# ProfitWell testing

In [121]:
import urllib
import requests # pip install requests 
import calendar;
import time;
import json

headers = {
  'Content-Type': 'application/json',
  'Authorization': '7f68c9b3acf02c17ab97e6681f51f071'
}

## new purchase testing

In [187]:
def create_sub_payload(user, plan, status='active'):
    payload = {
        "user_alias": user['id'],
        "email": user['email'],
        "plan_id": plan['id'],
        "plan_interval": plan['interval'],
        "plan_currency": plan['currency'],
        "status": status,
        "value": plan['price'],
        "effective_date": calendar.timegm(time.gmtime())
    }
    return payload

def update_sub_payload(plan, status='active'):
    payload = {
        'plan_id': plan['id'],
        'plan_interval': plan['interval'],
        'value': plan['price'],
        'status': status,
        "effective_date": calendar.timegm(time.gmtime())
    }
    return payload


# def make_single_payload(user, plan, status='active'):
#     payload = {
#         "user_alias": user['id'],
#         "subscription_alias": user['gateway_subscription_id'],
#         "email": user['email'],
#         "plan_id": plan['id'],
#         "plan_currency": plan['currency'],
#         "status": status,
#         "value": plan['price'],
#         "effective_date": calendar.timegm(time.gmtime()),
#         "plan_interval": 'n/a'
#     }
#     return payload

In [94]:
make_payload(user, plan, 'trialing')

{'user_alias': '2',
 'subscription_alias': '000003',
 'email': 'johnny2_first_buy@test.com',
 'plan_id': '1',
 'plan_interval': 'month',
 'plan_currency': 'usd',
 'status': 'trialing',
 'value': 5000,
 'effective_date': 1617704463}

### Test api status

In [195]:
r = requests.get('https://api.profitwell.com/v2/api-status/', headers=headers)
print(r.status_code, r.text)

200 


### Create subscription

In [89]:
user = {
    "id": "2",
    "email": "johnny2_first_buy@test.com",
    "gateway_subscription_id": '000003'
}

plan = {
    'id': "1",
    'interval': 'month',
    'currency': 'usd',
    'price': 5000
}

r = requests.post('https://api.profitwell.com/v2/subscriptions/', data = json.dumps(make_payload(user, plan)), headers=headers)
print(r.status_code, r.text)

201 {"user_id":"pwu_GRBMXQZOL4ZP","subscription_id":"pws_UCLaWCrBgblB","user_alias":"2","subscription_alias":"000003","email":"johnny2_first_buy@test.com","plan_id":"1","plan_interval":"month","plan_currency":"usd","status":"active","value":5000,"effective_date":1617677697,"meta":{}}


### churning subscription

In [92]:
gateway_subscription_id = '000003'
subscription_alians = gateway_subscription_id

r = requests.delete(f'https://api.profitwell.com/v2/subscriptions/{subscription_alians}/?effective_date={calendar.timegm(time.gmtime())}&churn_type=voluntary', headers=headers)
print(r.status_code, r.text)

400 {"non_field_errors":["This subscription is already scheduled to churn. To move the churn date, you must un-churn this subscription first"]}


In [194]:
## Get profitwell user info
user_alias = '21'

r = requests.get(f'https://api.profitwell.com/v2/users/{user_alias}', headers=headers)
print(r.status_code, r.text)

200 [{"user_id":"pwu_tswTJCFkat4A","subscription_id":"pws_hxoWpOSy0YFs","user_alias":"21","subscription_alias":null,"email":"year_cancel_user_21@test.com","plan_id":"12","plan_interval":"year","plan_currency":"usd","status":"active","value":9588,"effective_date":1618383645,"meta":{}},{"user_id":"pwu_tswTJCFkat4A","subscription_id":"pws_hxoWpOSy0YFs","user_alias":"21","subscription_alias":null,"email":"year_cancel_user_21@test.com","plan_id":"12","plan_interval":"year","plan_currency":"usd","status":"churned_voluntary","value":0,"effective_date":1620975683,"meta":{"churn_type":"voluntary"}}]


## testing scenario
1. trail -> month
1. trail -> month -> cancel
1. trail -> month -> discount
1. trail -> month -> discount -> cancel
1. trail -> cancel
1. year
1. year -> cancel
1. year -> discount
1. year -> discount -> cancel


### 1. trail -> month

In [148]:
# buy trail   [v]
user = {
    "id": "11",
    "email": "trail_to_month_user_11@test.com"
}

plan = {
    'id': "13",
    'interval': 'month',
    'currency': 'usd',
    'price': 0
}

r = requests.post('https://api.profitwell.com/v2/subscriptions/', data = json.dumps(create_sub_payload(user, plan, 'trialing')), headers=headers)
print(r.status_code, r.text)

201 {"user_id":"pwu_RetIOpd9GjRX","subscription_id":"pws_Ob6rvrlAwH7Q","user_alias":"11","subscription_alias":null,"email":"trail_to_month_user_11@test.com","plan_id":"13","plan_interval":"month","plan_currency":"usd","status":"trialing","value":0,"effective_date":1618212257,"meta":{}}


In [186]:
# trail renew to month  [v]
profitwell_sub_id = 'pws_Ob6rvrlAwH7Q'

switch_plan = {
    'id': "3",
    'interval': 'month',
    'price': 3995
}

r = requests.put(f'https://api.profitwell.com/v2/subscriptions/{profitwell_sub_id}/', data=json.dumps(update_sub_payload(switch_plan)), headers=headers)
print(r.status_code, r.text)

200 {"user_id":"pwu_RetIOpd9GjRX","subscription_id":"pws_Ob6rvrlAwH7Q","user_alias":"11","subscription_alias":null,"email":"trail_to_month_user_11@test.com","plan_id":"3","plan_interval":"month","plan_currency":"usd","status":"active","value":3995,"effective_date":1618368944,"meta":{}}


### 2. trail -> month -> cancel

In [149]:
# create trail  [v]
user = {
    "id": "12",
    "email": "trail_month_cancel_user_12@test.com"
}

plan = {
    'id': "13",
    'interval': 'month',
    'currency': 'usd',
    'price': 0
}

r = requests.post('https://api.profitwell.com/v2/subscriptions/', data = json.dumps(create_sub_payload(user, plan, 'trialing')), headers=headers)
print(r.status_code, r.text)

201 {"user_id":"pwu_mv7Uq9IuGIaB","subscription_id":"pws_mwcKQN15h9PR","user_alias":"12","subscription_alias":null,"email":"trail_month_cancel_user_12@test.com","plan_id":"13","plan_interval":"month","plan_currency":"usd","status":"trialing","value":0,"effective_date":1618212823,"meta":{}}


In [190]:
# trail renew to month  [v]
profitwell_sub_id = 'pws_mwcKQN15h9PR'

switch_plan = {
    'id': "3",
    'interval': 'month',
    'price': 3995
}

r = requests.put(f'https://api.profitwell.com/v2/subscriptions/{profitwell_sub_id}/', data=json.dumps(update_sub_payload(switch_plan)), headers=headers)
print(r.status_code, r.text)

200 {"user_id":"pwu_mv7Uq9IuGIaB","subscription_id":"pws_mwcKQN15h9PR","user_alias":"12","subscription_alias":null,"email":"trail_month_cancel_user_12@test.com","plan_id":"3","plan_interval":"month","plan_currency":"usd","status":"active","value":3995,"effective_date":1618383056,"meta":{}}


In [116]:
# month to cancel
profitwell_sub_id = 'pws_mwcKQN15h9PR'

r = requests.delete(f'https://api.profitwell.com/v2/subscriptions/{profitwell_sub_id}/?effective_date={calendar.timegm(time.gmtime())}&churn_type=voluntary', headers=headers)
print(r.status_code, r.text)

200 {"user_id":"pwu_3ndAMCl2gpr8","subscription_id":"pws_KljZdhe1NV6V","user_alias":"4","subscription_alias":"trail_to_month_id_4_plan_3","email":"trail_to_month_4@test.com","plan_id":"15","plan_interval":"month","plan_currency":"usd","status":"active","value":995,"effective_date":1617770348,"meta":{}}


### 3. trail -> month -> discount

In [152]:
# create trail  [v]
user = {
    "id": "14",
    "email": "trail_month_discount_user_14@test.com"
}

plan = {
    'id': "13",
    'interval': 'month',
    'currency': 'usd',
    'price': 0
}

r = requests.post('https://api.profitwell.com/v2/subscriptions/', data = json.dumps(create_sub_payload(user, plan, 'trialing')), headers=headers)
print(r.status_code, r.text)

201 {"user_id":"pwu_EGSlu9AkfB34","subscription_id":"pws_88TvsMtiTCzc","user_alias":"14","subscription_alias":null,"email":"trail_month_discount_user_14@test.com","plan_id":"13","plan_interval":"month","plan_currency":"usd","status":"trialing","value":0,"effective_date":1618213354,"meta":{}}


In [184]:
# trail renew to month  [v]
profitwell_sub_id = 'pws_88TvsMtiTCzc'

switch_plan = {
    'id': "3",
    'interval': 'month',
    'price': 3995
}

r = requests.put(f'https://api.profitwell.com/v2/subscriptions/{profitwell_sub_id}/', data=json.dumps(update_sub_payload(switch_plan)), headers=headers)
print(r.status_code, r.text)

200 {"user_id":"pwu_EGSlu9AkfB34","subscription_id":"pws_88TvsMtiTCzc","user_alias":"14","subscription_alias":null,"email":"trail_month_discount_user_14@test.com","plan_id":"3","plan_interval":"month","plan_currency":"usd","status":"active","value":3995,"effective_date":1618368927,"meta":{}}


In [191]:
# month to discount  [v]
profitwell_sub_id = 'pws_88TvsMtiTCzc'

switch_plan = {
    'id': "16",
    'interval': 'month',
    'price': 495
}

r = requests.put(f'https://api.profitwell.com/v2/subscriptions/{profitwell_sub_id}/', data=json.dumps(update_sub_payload(switch_plan)), headers=headers)
print(r.status_code, r.text)

200 {"user_id":"pwu_EGSlu9AkfB34","subscription_id":"pws_88TvsMtiTCzc","user_alias":"14","subscription_alias":null,"email":"trail_month_discount_user_14@test.com","plan_id":"16","plan_interval":"month","plan_currency":"usd","status":"active","value":495,"effective_date":1618383153,"meta":{}}


### 4. trail -> month -> discount -> cancel

In [153]:
# create trail  [v]
user = {
    "id": "15",
    "email": "trail_month_discount_cancel_user_15@test.com"
}

plan = {
    'id': "13",
    'interval': 'month',
    'currency': 'usd',
    'price': 0
}

r = requests.post('https://api.profitwell.com/v2/subscriptions/', data = json.dumps(create_sub_payload(user, plan, 'trialing')), headers=headers)
print(r.status_code, r.text)

201 {"user_id":"pwu_b4dRVI9m0aqq","subscription_id":"pws_vOqy1mIc1gVn","user_alias":"15","subscription_alias":null,"email":"trail_month_discount_cancel_user_15@test.com","plan_id":"13","plan_interval":"month","plan_currency":"usd","status":"trialing","value":0,"effective_date":1618213377,"meta":{}}


In [183]:
# trail renew to month  [v]
profitwell_sub_id = 'pws_vOqy1mIc1gVn'

switch_plan = {
    'id': "3",
    'interval': 'month',
    'price': 3995
}

r = requests.put(f'https://api.profitwell.com/v2/subscriptions/{profitwell_sub_id}/', data=json.dumps(update_sub_payload(switch_plan)), headers=headers)
print(r.status_code, r.text)

200 {"user_id":"pwu_b4dRVI9m0aqq","subscription_id":"pws_vOqy1mIc1gVn","user_alias":"15","subscription_alias":null,"email":"trail_month_discount_cancel_user_15@test.com","plan_id":"3","plan_interval":"month","plan_currency":"usd","status":"active","value":3995,"effective_date":1618368919,"meta":{}}


In [ ]:
# month to dscount
profitwell_sub_id = 'pws_mwcKQN15h9PR'

switch_plan = {
    'plan_id': "16",
    'interval': 'month',
    'value': 495
}

r = requests.put(f'https://api.profitwell.com/v2/subscriptions/{profitwell_sub_id}/', data=json.dumps(update_sub_payload(switch_plan)), headers=headers)
print(r.status_code, r.text)

In [ ]:
# month to cancel
profitwell_sub_id = 'pws_mwcKQN15h9PR'
ge
r = requests.delete(f'https://api.profitwell.com/v2/subscriptions/{profitwell_sub_id}/?effective_date={calendar.timegm(time.gmtime())}&churn_type=voluntary', headers=headers)
print(r.status_code, r.text)

### 5. trail -> cancel

In [154]:
# create trail  [v]
user = {
    "id": "16",
    "email": "trail_cancel_user_16@test.com"
}

plan = {
    'id': "13",
    'interval': 'month',
    'currency': 'usd',
    'price': 0
}

r = requests.post('https://api.profitwell.com/v2/subscriptions/', data = json.dumps(create_sub_payload(user, plan, 'trialing')), headers=headers)
print(r.status_code, r.text)

201 {"user_id":"pwu_tc2CJ2i8Vmix","subscription_id":"pws_7YHgENYXEtVS","user_alias":"16","subscription_alias":null,"email":"trail_cancel_user_16@test.com","plan_id":"13","plan_interval":"month","plan_currency":"usd","status":"trialing","value":0,"effective_date":1618213571,"meta":{}}


In [174]:
# trail to cancel  [v]
profitwell_sub_id = 'pws_7YHgENYXEtVS'

r = requests.delete(f'https://api.profitwell.com/v2/subscriptions/{profitwell_sub_id}/?effective_date={calendar.timegm(time.gmtime())}&churn_type=voluntary', headers=headers)
print(r.status_code, r.text)

200 {"user_id":"pwu_tc2CJ2i8Vmix","subscription_id":"pws_7YHgENYXEtVS","user_alias":"16","subscription_alias":null,"email":"trail_cancel_user_16@test.com","plan_id":"13","plan_interval":"month","plan_currency":"usd","status":"churned_voluntary","value":0,"effective_date":1618361531,"meta":{"churn_type":"voluntary"}}


### 6. year

In [159]:
# create year  [v]
user = {
    "id": "17",
    "email": "year_user_17@test.com"
}

plan = {
    'id': "12",
    'interval': 'year',
    'currency': 'usd',
    'price': 9588
}

r = requests.post('https://api.profitwell.com/v2/subscriptions/', data = json.dumps(create_sub_payload(user, plan)), headers=headers)
print(r.status_code, r.text)

201 {"user_id":"pwu_nzyrrfojLfyn","subscription_id":"pws_LmmobnjvYZGQ","user_alias":"17","subscription_alias":null,"email":"year_user_17@test.com","plan_id":"12","plan_interval":"year","plan_currency":"usd","status":"active","value":9588,"effective_date":1618214449,"meta":{}}


### 7. year -> cancel

In [192]:
# create year  [v]
user = {
    "id": "21",
    "email": "year_cancel_user_21@test.com"
}

plan = {
    'id': "12",
    'interval': 'year',
    'currency': 'usd',
    'price': 9588
}

r = requests.post('https://api.profitwell.com/v2/subscriptions/', data = json.dumps(create_sub_payload(user, plan)), headers=headers)
print(r.status_code, r.text)

201 {"user_id":"pwu_tswTJCFkat4A","subscription_id":"pws_hxoWpOSy0YFs","user_alias":"21","subscription_alias":null,"email":"year_cancel_user_21@test.com","plan_id":"12","plan_interval":"year","plan_currency":"usd","status":"active","value":9588,"effective_date":1618383645,"meta":{}}


In [175]:
# year to cancel
profitwell_sub_id = 'pws_hxoWpOSy0YFs'

r = requests.delete(f'https://api.profitwell.com/v2/subscriptions/{profitwell_sub_id}/?effective_date={calendar.timegm(time.gmtime())}&churn_type=voluntary', headers=headers)
print(r.status_code, r.text)

200 {"user_id":"pwu_ofps6zGS4p54","subscription_id":"pws_qnXdrhbxoZV4","user_alias":"18","subscription_alias":null,"email":"year_cancel_user_18@test.com","plan_id":"12","plan_interval":"year","plan_currency":"usd","status":"churned_voluntary","value":0,"effective_date":1618361630,"meta":{"churn_type":"voluntary"}}


### 8. year -> discount

In [161]:
# create year  [v]
user = {
    "id": "19",
    "email": "year_discount_user_19@test.com"
}

plan = {
    'id': "12",
    'interval': 'year',
    'currency': 'usd',
    'price': 9588
}

r = requests.post('https://api.profitwell.com/v2/subscriptions/', data = json.dumps(create_sub_payload(user, plan)), headers=headers)
print(r.status_code, r.text)

201 {"user_id":"pwu_lRYIxINJcs5h","subscription_id":"pws_WmPjk9ARZsRs","user_alias":"19","subscription_alias":null,"email":"year_discount_user_19@test.com","plan_id":"12","plan_interval":"year","plan_currency":"usd","status":"active","value":9588,"effective_date":1618214739,"meta":{}}


In [188]:
# year to dscount [v]
profitwell_sub_id = 'pws_WmPjk9ARZsRs'

switch_plan = {
    'id': "16",
    'interval': 'month',
    'price': 495
}

r = requests.put(f'https://api.profitwell.com/v2/subscriptions/{profitwell_sub_id}/', data=json.dumps(update_sub_payload(switch_plan)), headers=headers)
print(r.status_code, r.text)

200 {"user_id":"pwu_lRYIxINJcs5h","subscription_id":"pws_WmPjk9ARZsRs","user_alias":"19","subscription_alias":null,"email":"year_discount_user_19@test.com","plan_id":"16","plan_interval":"month","plan_currency":"usd","status":"active","value":495,"effective_date":1618374039,"meta":{}}


### 9. year -> discount -> cancel

In [163]:
# create year  [v]
user = {
    "id": "20",
    "email": "year_discount_cancle_user_20@test.com"
}

plan = {
    'id': "12",
    'interval': 'year',
    'currency': 'usd',
    'price': 9588
}

r = requests.post('https://api.profitwell.com/v2/subscriptions/', data = json.dumps(create_sub_payload(user, plan)), headers=headers)
print(r.status_code, r.text)

201 {"user_id":"pwu_6DfE5oLLNA6Y","subscription_id":"pws_RqYP51rr54EG","user_alias":"20","subscription_alias":null,"email":"year_discount_cancle_user_20@test.com","plan_id":"12","plan_interval":"year","plan_currency":"usd","status":"active","value":9588,"effective_date":1618214792,"meta":{}}


In [189]:
# year to dscount
profitwell_sub_id = 'pws_mwcKQN15h9PR'

switch_plan = {
    'id': "16",
    'interval': 'month',
    'price': 495
}

r = requests.put(f'https://api.profitwell.com/v2/subscriptions/{profitwell_sub_id}/', data=json.dumps(update_sub_payload(switch_plan)), headers=headers)
print(r.status_code, r.text)

200 {"user_id":"pwu_mv7Uq9IuGIaB","subscription_id":"pws_mwcKQN15h9PR","user_alias":"12","subscription_alias":null,"email":"trail_month_cancel_user_12@test.com","plan_id":"16","plan_interval":"month","plan_currency":"usd","status":"active","value":495,"effective_date":1618374055,"meta":{}}


In [ ]:
# year to cancel
profitwell_sub_id = 'pws_mwcKQN15h9PR'

r = requests.delete(f'https://api.profitwell.com/v2/subscriptions/{profitwell_sub_id}/?effective_date={calendar.timegm(time.gmtime())}&churn_type=voluntary', headers=headers)
print(r.status_code, r.text)

### test add trait

In [200]:
pyalod = {
            "email": "email1@legaltemplates.net",
            "category": "card_type",
            "trait": "visa2"
        }
r = requests.put('https://api.profitwell.com/v2/customer_traits/trait/', data = json.dumps(pyalod), headers=headers)
print(r.status_code, r.text)

400 {"params":"Customer already has trait (category: 'card_type', trait: 'visa2')"}


In [166]:

for i in range(1, 5):
    my_id = f"trait_test_{i}_{calendar.timegm(time.gmtime())}"
    # create year  [v]
    user = {
        "id": my_id,
        "email": f"{my_id}@test.com"
    }

    plan = {
        'id': "12",
        'interval': 'year',
        'currency': 'usd',
        'price': 9588
    }

    r = requests.post('https://api.profitwell.com/v2/subscriptions/', data = json.dumps(create_sub_payload(user, plan)), headers=headers)
    print(r.status_code, r.text)

    traits = [
        {
            "email": f"{my_id}@test.com",
            "category": "card type",
            "trait": "visa"
        },
        {
            "email": f"{my_id}@test.com",
            "category": "first form", #what is first form
            "trait": "johnny form name"
        },
        {
            "email": f"{my_id}@test.com",
            "category": "gateway name",
            "trait": "ccbill_internal_eu"
        },
        {
            "email": f"{my_id}@test.com",
            "category": "utm source", #what is utm source
            "trait": "source xxx"
        },
    ]
    for trait in traits:
        r = requests.put('https://api.profitwell.com/v2/customer_traits/trait/', data = json.dumps(trait), headers=headers)
        print(r.status_code, r.text)


201 {"user_id":"pwu_pR36J2RpxTqm","subscription_id":"pws_Tehm55izB9CI","user_alias":"trait_test_1_1618215419","subscription_alias":null,"email":"trait_test_1_1618215419@test.com","plan_id":"12","plan_interval":"year","plan_currency":"usd","status":"active","value":9588,"effective_date":1618215419,"meta":{}}
204 
204 
204 
204 
201 {"user_id":"pwu_sHXQCg6dZvfu","subscription_id":"pws_Dxz1fLoozHVA","user_alias":"trait_test_2_1618215423","subscription_alias":null,"email":"trait_test_2_1618215423@test.com","plan_id":"12","plan_interval":"year","plan_currency":"usd","status":"active","value":9588,"effective_date":1618215423,"meta":{}}
204 
204 
204 
204 
201 {"user_id":"pwu_xBuJ99ZuDlf8","subscription_id":"pws_jWj24NFyl8ei","user_alias":"trait_test_3_1618215426","subscription_alias":null,"email":"trait_test_3_1618215426@test.com","plan_id":"12","plan_interval":"year","plan_currency":"usd","status":"active","value":9588,"effective_date":1618215426,"meta":{}}
204 
204 
204 
204 
201 {"user_id

### single document

In [118]:
user = {
    "id": "8",
    "email": "single_8@test.com",
    "gateway_subscription_id": 'single_id_8_plan_4'
}

plan = {
    'id': "4",
    'currency': 'usd',
    'price': 5995
}

r = requests.post('https://api.profitwell.com/v2/subscriptions/', data = json.dumps(make_single_payload(user, plan)), headers=headers)
print(r.status_code, r.text)

400 {"plan_interval":["This field is required."]}


### new subscription -< cancel

In [138]:
user = {
    "id": "9",
    "email": "trail_to_cancel_9@test.com",
    "gateway_subscription_id": 'sub_9'
}

plan = {
    'id': "3",
    'interval': 'month',
    'currency': 'usd',
    'price': 3995
}

r = requests.post('https://api.profitwell.com/v2/subscriptions/', data = json.dumps(make_payload(user, plan)), headers=headers)
print(r.status_code, r.text)

400 {"non_field_errors":["Subscription pws_EAIG75d3rsjO already exists"]}


In [137]:
user_alias = '9'

r = requests.get(f'https://api.profitwell.com/v2/users/{user_alias}', headers=headers)
print(r.status_code, r.text)

200 [{"user_id":"pwu_QtZk0DLx2Vri","subscription_id":"pws_EAIG75d3rsjO","user_alias":"9","subscription_alias":"sub_9","email":"trail_to_cancel_9@test.com","plan_id":"3","plan_interval":"month","plan_currency":"usd","status":"active","value":3995,"effective_date":1618192241,"meta":{}},{"user_id":"pwu_QtZk0DLx2Vri","subscription_id":"pws_EAIG75d3rsjO","user_alias":"9","subscription_alias":"sub_9","email":"trail_to_cancel_9@test.com","plan_id":"3","plan_interval":"month","plan_currency":"usd","status":"churned_voluntary","value":0,"effective_date":1618196084,"meta":{"churn_type":"voluntary"}}]


In [136]:
# trail to cancel
gateway_subscription_id = 'sub_9'
subscription_alians = gateway_subscription_id

r = requests.delete(f'https://api.profitwell.com/v2/subscriptions/{subscription_alians}/?effective_date={calendar.timegm(time.gmtime())}&churn_type=voluntary', headers=headers)
print(r.status_code, r.text)

200 {"user_id":"pwu_QtZk0DLx2Vri","subscription_id":"pws_EAIG75d3rsjO","user_alias":"9","subscription_alias":"sub_9","email":"trail_to_cancel_9@test.com","plan_id":"3","plan_interval":"month","plan_currency":"usd","status":"churned_voluntary","value":0,"effective_date":1618196084,"meta":{"churn_type":"voluntary"}}
